# AQC per il problema dello zaino

In [1]:
import math
import pandas as pd
import dimod
import sympy as sp
from sympy.abc import m, M, c, lamda
import dwave.system as dw

/var/folders/_7/n7t6c5_x7f17npn6_q_r18gh0000gn/T/ipykernel_3747/709188333.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/giulia/tesinaOC/ocean/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 

In [2]:
data = pd.read_csv('data/small.csv', names=['p', 'w'])
data.index = range(1, len(data) + 1)

data

,p,w
1,35,12
2,85,27
3,30,11
4,50,17
5,70,20
6,80,10
7,55,15


In [3]:
num_items = len(data)
capacity = int(0.8 * sum(data['w'])) # Fissa capacità dello zaino all'80% della somma dei pesi degli elementi

print(f'c = {capacity}')

c = 89


### Prima formulazione

In [40]:
j = sp.Idx('j')         # Indici per le sommatorie
n = sp.Idx('n')
X = sp.IndexedBase('x') # Vettori variabili oggetti
P = sp.IndexedBase('p') # Vettori variabili profitto
W = sp.IndexedBase('w') # Vettori variabili pesi
Y = sp.IndexedBase('y') # Vettori variabili di slack

syms = [X[k] for k in range(1, num_items + 1)] + [Y[k] for k in range(1, capacity + 1)]

In [41]:
first_f = -sp.Sum(P[j] * X[j], (j, 1, m)) + \
          lamda * ((1 - sp.Sum(Y[n], (n, 1, c)))**2 +
                   (sp.Sum(n * Y[n], (n, 1, c)) - sp.Sum(W[j] * X[j], (j, 1, m)))**2)
first_f

lamda*((1 - Sum(y[n], (n, 1, c)))**2 + (Sum(n*y[n], (n, 1, c)) - Sum(w[j]*x[j], (j, 1, m)))**2) - Sum(p[j]*x[j], (j, 1, m))

In [42]:
first_f = first_f.subs([(m, num_items), (c, capacity)]).doit() \
                 .subs([(P[i], data['p'][i]) for i in range(1, num_items + 1)]) \
                 .subs([(W[i], data['w'][i]) for i in range(1, num_items + 1)]) \
                 .subs(lamda, 1 + data['p'].max())
first_f

86*(-y[10] - y[11] - y[12] - y[13] - y[14] - y[15] - y[16] - y[17] - y[18] - y[19] - y[1] - y[20] - y[21] - y[22] - y[23] - y[24] - y[25] - y[26] - y[27] - y[28] - y[29] - y[2] - y[30] - y[31] - y[32] - y[33] - y[34] - y[35] - y[36] - y[37] - y[38] - y[39] - y[3] - y[40] - y[41] - y[42] - y[43] - y[44] - y[45] - y[46] - y[47] - y[48] - y[49] - y[4] - y[50] - y[51] - y[52] - y[53] - y[54] - y[55] - y[56] - y[57] - y[58] - y[59] - y[5] - y[60] - y[61] - y[62] - y[63] - y[64] - y[65] - y[66] - y[67] - y[68] - y[69] - y[6] - y[70] - y[71] - y[72] - y[73] - y[74] - y[75] - y[76] - y[77] - y[78] - y[79] - y[7] - y[80] - y[81] - y[82] - y[83] - y[84] - y[85] - y[86] - y[87] - y[88] - y[89] - y[8] - y[9] + 1)**2 + 86*(-12*x[1] - 27*x[2] - 11*x[3] - 17*x[4] - 20*x[5] - 10*x[6] - 15*x[7] + 10*y[10] + 11*y[11] + 12*y[12] + 13*y[13] + 14*y[14] + 15*y[15] + 16*y[16] + 17*y[17] + 18*y[18] + 19*y[19] + y[1] + 20*y[20] + 21*y[21] + 22*y[22] + 23*y[23] + 24*y[24] + 25*y[25] + 26*y[26] + 27*y[27] + 28*y

In [43]:
first_f = sp.expand(first_f).replace(lambda x: x.is_Pow, lambda x: x.base); first_f

55728*x[1]*x[2] + 22704*x[1]*x[3] + 35088*x[1]*x[4] + 41280*x[1]*x[5] + 20640*x[1]*x[6] + 30960*x[1]*x[7] - 20640*x[1]*y[10] - 22704*x[1]*y[11] - 24768*x[1]*y[12] - 26832*x[1]*y[13] - 28896*x[1]*y[14] - 30960*x[1]*y[15] - 33024*x[1]*y[16] - 35088*x[1]*y[17] - 37152*x[1]*y[18] - 39216*x[1]*y[19] - 2064*x[1]*y[1] - 41280*x[1]*y[20] - 43344*x[1]*y[21] - 45408*x[1]*y[22] - 47472*x[1]*y[23] - 49536*x[1]*y[24] - 51600*x[1]*y[25] - 53664*x[1]*y[26] - 55728*x[1]*y[27] - 57792*x[1]*y[28] - 59856*x[1]*y[29] - 4128*x[1]*y[2] - 61920*x[1]*y[30] - 63984*x[1]*y[31] - 66048*x[1]*y[32] - 68112*x[1]*y[33] - 70176*x[1]*y[34] - 72240*x[1]*y[35] - 74304*x[1]*y[36] - 76368*x[1]*y[37] - 78432*x[1]*y[38] - 80496*x[1]*y[39] - 6192*x[1]*y[3] - 82560*x[1]*y[40] - 84624*x[1]*y[41] - 86688*x[1]*y[42] - 88752*x[1]*y[43] - 90816*x[1]*y[44] - 92880*x[1]*y[45] - 94944*x[1]*y[46] - 97008*x[1]*y[47] - 99072*x[1]*y[48] - 101136*x[1]*y[49] - 8256*x[1]*y[4] - 103200*x[1]*y[50] - 105264*x[1]*y[51] - 107328*x[1]*y[52] - 109

In [44]:
first_Q = pd.DataFrame(0, index=syms, columns=syms)

first_p = sp.Poly(first_f)  # Viene interpretato come un polinomio

for sym1 in syms:
    for sym2 in syms[syms.index(sym1):]:
        first_Q.loc[sym1, sym2] = int(first_p.coeff_monomial(sym1 if sym1 == sym2 else sym1*sym2))

first_Q

,x[1],x[2],x[3],x[4],x[5],x[6],x[7],y[1],y[2],y[3],...,y[80],y[81],y[82],y[83],y[84],y[85],y[86],y[87],y[88],y[89]
x[1],12349,55728,22704,35088,41280,20640,30960,-2064,-4128,-6192,...,-165120,-167184,-169248,-171312,-173376,-175440,-177504,-179568,-181632,-183696
x[2],0,62609,51084,78948,92880,46440,69660,-4644,-9288,-13932,...,-371520,-376164,-380808,-385452,-390096,-394740,-399384,-404028,-408672,-413316
x[3],0,0,10376,32164,37840,18920,28380,-1892,-3784,-5676,...,-151360,-153252,-155144,-157036,-158928,-160820,-162712,-164604,-166496,-168388
x[4],0,0,0,24804,58480,29240,43860,-2924,-5848,-8772,...,-233920,-236844,-239768,-242692,-245616,-248540,-251464,-254388,-257312,-260236
x[5],0,0,0,0,34330,34400,51600,-3440,-6880,-10320,...,-275200,-278640,-282080,-285520,-288960,-292400,-295840,-299280,-302720,-306160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
y[85],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,621264,1257492,1272112,1286732,1301352
y[86],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,635970,1287076,1301868,1316660
y[87],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,650848,1317004,1331968
y[88],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,665898,1347276


brute force

In [45]:
dimod.ExactSolver().sample_qubo(first_Q).to_pandas_dataframe().sort_values(by='energy')

ValueError: Maximum allowed dimension exceeded

In [ ]:
dimod.SimulatedAnnealingSampler().sample_qubo(first_Q).to_pandas_dataframe().sort_values(by='energy')

### Seconda formulazione

In [4]:
i = sp.Idx('i')
j = sp.Idx('j')
n = sp.Idx('n')
X = sp.IndexedBase('x')
P = sp.IndexedBase('p')
W = sp.IndexedBase('w')
Y = sp.IndexedBase('y')

syms = [X[k] for k in range(1, num_items + 1)] + [Y[k] for k in range(0, math.floor(math.log(capacity, 2)) + 1)]

In [5]:
second_f = -sp.Sum(P[j] * X[j], (j, 1, m)) + \
           lamda * (((c - (2**M - 1)) * Y[M] + sp.Sum(2**i * Y[i], (i, 0, M - 1)) -
                     sp.Sum(W[j] * X[j], (j, 1, m)))**2)
second_f

lamda*((-2**M + c + 1)*y[M] + Sum(2**i*y[i], (i, 0, M - 1)) - Sum(w[j]*x[j], (j, 1, m)))**2 - Sum(p[j]*x[j], (j, 1, m))

In [6]:
second_f = second_f.subs([(m, num_items), (c, capacity), (M, math.floor(math.log(capacity, 2)))]).doit() \
                   .subs([(P[i], data['p'][i]) for i in range(1, num_items + 1)]) \
                   .subs([(W[i], data['w'][i]) for i in range(1, num_items + 1)]) \
                   .subs(lamda, 1 + data['p'].max())

second_f

86*(-12*x[1] - 27*x[2] - 11*x[3] - 17*x[4] - 20*x[5] - 10*x[6] - 15*x[7] + y[0] + 2*y[1] + 4*y[2] + 8*y[3] + 16*y[4] + 32*y[5] + 26*y[6])**2 - 35*x[1] - 85*x[2] - 30*x[3] - 50*x[4] - 70*x[5] - 80*x[6] - 55*x[7]

In [7]:
second_f = second_f.expand().replace(lambda x: x.is_Pow, lambda x: x.base); second_f

55728*x[1]*x[2] + 22704*x[1]*x[3] + 35088*x[1]*x[4] + 41280*x[1]*x[5] + 20640*x[1]*x[6] + 30960*x[1]*x[7] - 2064*x[1]*y[0] - 4128*x[1]*y[1] - 8256*x[1]*y[2] - 16512*x[1]*y[3] - 33024*x[1]*y[4] - 66048*x[1]*y[5] - 53664*x[1]*y[6] + 12349*x[1] + 51084*x[2]*x[3] + 78948*x[2]*x[4] + 92880*x[2]*x[5] + 46440*x[2]*x[6] + 69660*x[2]*x[7] - 4644*x[2]*y[0] - 9288*x[2]*y[1] - 18576*x[2]*y[2] - 37152*x[2]*y[3] - 74304*x[2]*y[4] - 148608*x[2]*y[5] - 120744*x[2]*y[6] + 62609*x[2] + 32164*x[3]*x[4] + 37840*x[3]*x[5] + 18920*x[3]*x[6] + 28380*x[3]*x[7] - 1892*x[3]*y[0] - 3784*x[3]*y[1] - 7568*x[3]*y[2] - 15136*x[3]*y[3] - 30272*x[3]*y[4] - 60544*x[3]*y[5] - 49192*x[3]*y[6] + 10376*x[3] + 58480*x[4]*x[5] + 29240*x[4]*x[6] + 43860*x[4]*x[7] - 2924*x[4]*y[0] - 5848*x[4]*y[1] - 11696*x[4]*y[2] - 23392*x[4]*y[3] - 46784*x[4]*y[4] - 93568*x[4]*y[5] - 76024*x[4]*y[6] + 24804*x[4] + 34400*x[5]*x[6] + 51600*x[5]*x[7] - 3440*x[5]*y[0] - 6880*x[5]*y[1] - 13760*x[5]*y[2] - 27520*x[5]*y[3] - 55040*x[5]*y[4] - 1100

In [8]:
second_Q = pd.DataFrame(0, index=syms, columns=syms)

second_p = sp.Poly(second_f)

for sym1 in syms:
    for sym2 in syms[syms.index(sym1):]:
            second_Q.loc[sym1, sym2] = int(second_p.coeff_monomial(sym1 if sym1 == sym2 else sym1*sym2))

second_Q

,x[1],x[2],x[3],x[4],x[5],x[6],x[7],y[0],y[1],y[2],y[3],y[4],y[5],y[6]
x[1],12349,55728,22704,35088,41280,20640,30960,-2064,-4128,-8256,-16512,-33024,-66048,-53664
x[2],0,62609,51084,78948,92880,46440,69660,-4644,-9288,-18576,-37152,-74304,-148608,-120744
x[3],0,0,10376,32164,37840,18920,28380,-1892,-3784,-7568,-15136,-30272,-60544,-49192
x[4],0,0,0,24804,58480,29240,43860,-2924,-5848,-11696,-23392,-46784,-93568,-76024
x[5],0,0,0,0,34330,34400,51600,-3440,-6880,-13760,-27520,-55040,-110080,-89440
x[6],0,0,0,0,0,8520,25800,-1720,-3440,-6880,-13760,-27520,-55040,-44720
x[7],0,0,0,0,0,0,19295,-2580,-5160,-10320,-20640,-41280,-82560,-67080
y[0],0,0,0,0,0,0,0,86,344,688,1376,2752,5504,4472
y[1],0,0,0,0,0,0,0,0,344,1376,2752,5504,11008,8944
y[2],0,0,0,0,0,0,0,0,0,1376,5504,11008,22016,17888


In [9]:
dimod.ExactSolver().sample_qubo(second_Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,energy,num_occurrences
10924,0,1,0,1,1,1,1,1,1,1,1,1,1,1,-340.0,1
10658,1,1,0,0,1,1,1,0,1,0,1,1,1,1,-325.0,1
10404,0,1,1,0,1,1,1,1,0,0,1,1,1,1,-320.0,1
10582,1,0,1,1,1,1,1,1,1,0,1,1,1,1,-320.0,1
11218,1,1,0,1,1,1,0,0,0,1,1,1,1,1,-320.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,1,1,1,1,1,1,1,0,0,1,0,0,0,0,1002699.0,1
341,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1021361.0,1
426,1,1,1,1,1,1,1,0,1,0,0,0,0,0,1040195.0,1
170,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1059201.0,1


In [10]:
dimod.SimulatedAnnealingSampler().sample_qubo(second_Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,energy,num_occurrences
4,1,1,1,1,0,1,0,1,1,0,0,1,1,1,-280.0,1
9,0,0,1,0,1,1,1,0,0,0,1,1,1,0,-235.0,1
0,1,0,1,1,1,1,1,0,0,1,1,1,1,1,-234.0,1
5,1,1,0,0,1,0,0,1,1,0,1,1,1,0,-190.0,1
1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,-180.0,1
2,0,1,1,1,0,0,0,1,1,1,0,1,1,0,-165.0,1
3,1,1,1,0,1,0,0,1,1,0,1,0,1,1,-134.0,1
6,0,0,0,0,1,0,1,1,0,0,1,0,0,1,-125.0,1
8,1,0,1,0,0,1,1,1,1,1,1,0,1,0,-114.0,1
7,0,0,0,0,1,0,0,1,1,1,1,0,0,0,2080.0,1


In [24]:
bqm = dimod.BinaryQuadraticModel.from_qubo(second_Q) 
qsampler = dw.EmbeddingComposite(dw.DWaveSampler()) 
qsampler.sample(bqm, num_reads=1000).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,chain_break_fraction,energy,num_occurrences
139,1,1,0,1,0,1,1,1,1,1,0,1,1,1,0.000000,-305.0,1
140,0,1,0,1,0,1,1,1,1,0,1,0,1,1,0.000000,-270.0,1
141,0,1,0,1,1,1,1,0,1,1,1,1,1,1,0.000000,-254.0,1
142,0,1,1,0,0,1,1,1,0,1,0,0,1,1,0.000000,-250.0,1
143,0,1,0,0,1,1,0,1,0,0,1,1,1,0,0.000000,-235.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,0,1,1,0,1,0,0,1,1,0,1,1,0,0,0.071429,82461.0,2
135,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0.071429,82491.0,1
72,0,1,0,0,1,1,0,1,0,0,1,1,0,0,0.071429,87829.0,1
115,1,1,1,1,0,1,1,0,0,0,0,0,1,1,0.071429,99081.0,1
